In [1]:
from IPython.display import display
import ipywidgets as widgets

In [3]:
# Importar bibliotecas necesarias
import os
from dotenv import load_dotenv
import google.generativeai as genai
from IPython.display import display
import ipywidgets as widgets

# Cargar variables de entorno desde .env
load_dotenv()



# Configurar el modelo
def inicializar_modelo():
    api_key = "tu-api-key-gemini"
    if api_key:
        genai.configure(api_key=api_key)
        return genai.GenerativeModel('gemini-pro')
    else:
        print("❌ No se pudo configurar el modelo debido a un error de autenticación.")
        return None

# Gestionar el historial de conversaciones
historial = []

def gestionar_historial():
    global historial
    if len(historial) > 4:
        historial = historial[-4:]

# Generar respuesta del Dungeon Master
def generar_respuesta(prompt):
    model = inicializar_modelo()
    if not model:
        return "❌ Error: Modelo no inicializado."
    
    gestionar_historial()
    contexto = "\n".join(
        [f"{msg['role']}: {msg['content']}" for msg in historial[-2:]]
    )
    prompt_estructurado = f"""
    [ROL]
    Eres un Dungeon Master de D&D 5e experto. Reglas:
    1. Mantén coherencia con el escenario actual
    2. Máximo 3 párrafos breves
    3. Nunca asumas acciones del jugador
    4. Progresa la historia gradualmente
    [CONTEXTO]
    {contexto}
    [ACCIÓN DEL JUGADOR]
    {prompt}
    [RESPUESTA DM]
    • Describe consecuencias lógicas
    • Mantén el ambiente establecido
    • Usa diálogos NPC cuando sea relevante
    • Incluye posibles caminos de acción
    """
    try:
        response = model.generate_content(prompt_estructurado)
        return response.text
    except Exception as e:
        return f"⚡ Error en la generación: {str(e)}"

# Interfaz interactiva
def main():
    print("🎲 D&D AI Dungeon Master")
    output = widgets.Output()
    input_box = widgets.Text(placeholder="¿Qué hace tu personaje?")
    send_button = widgets.Button(description="Enviar")
    
    def on_send_button_clicked(_):
        global historial
        with output:
            # Obtener la entrada del usuario
            prompt = input_box.value.strip()
            if not prompt:
                print("⚠️ Por favor, ingresa una acción.")
                return
            
            # Añadir mensaje del usuario al historial
            historial.append({"role": "user", "content": prompt})
            print(f"\n👤 Usuario: {prompt}")
            
            # Generar respuesta del Dungeon Master
            with output:
                print("\n🎭 El Dungeon Master está pensando...")
            respuesta = generar_respuesta(prompt)
            historial.append({"role": "assistant", "content": respuesta})
            print(f"\n🧙‍♂️ Dungeon Master: {respuesta}")
            input_box.value = ""
    
    send_button.on_click(on_send_button_clicked)
    
    # Mostrar la interfaz
    display(widgets.VBox([input_box, send_button, output]))

# Ejecutar la aplicación
if __name__ == "__main__":
    main()

🎲 D&D AI Dungeon Master
